In [1]:
import pandas as pd
import numpy as np
import implicit
import scipy.sparse as sp

c:\Users\benja\Documents\Universidad\2025-2\Recomendadores\Proyecto\.venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Metricas

In [2]:
df_train = pd.read_csv(
    "train", sep=",", names=["userid", "itemid", "rating"], header=None
)

df_train.rating = [1 if x >= 5 else 0 for x in df_train.rating]

df_train.head()

,userid,itemid,rating
0,1635,6675.0,1
1,3340,7785.0,1
2,1015,578.0,1
3,87,18617.0,1
4,5582,8425.0,1


In [3]:
df_test = pd.read_csv(
    "test", sep=",", names=["userid", "itemid", "rating"], header=None
)

df_test.head()

,userid,itemid,rating
0,20408,8426.0,8
1,22049,5114.0,10
2,4728,4224.0,10
3,690,6746.0,9
4,254,481.0,6


In [4]:
user_items = {}
itemset = set()

for row in df_train.itertuples():
    if row[1] not in user_items:
        user_items[row[1]] = []

    user_items[row[1]].append(row[2])
    itemset.add(row[2])

itemset = np.sort(list(itemset))

sparse_matrix = np.zeros((len(user_items), len(itemset)))

for i, items in enumerate(user_items.values()):
    sparse_matrix[i] = np.isin(itemset, items, assume_unique=True).astype(int)

matrix = sp.csr_matrix(sparse_matrix.T)

user_item_matrix = matrix.T.tocsr()

In [5]:
user2row = {user_id: matrix_row for matrix_row, user_id in enumerate(user_items.keys())}

item2col = {item_id: matrix_col for matrix_col, item_id in enumerate(itemset)}
col2item = {matrix_col: item_id for item_id, matrix_col in item2col.items()}

In [6]:
model_als = implicit.als.AlternatingLeastSquares(factors=300)
model_als.fit(user_item_matrix, show_progress=False)

c:\Users\benja\Documents\Universidad\2025-2\Recomendadores\Proyecto\.venv\lib\site-packages\implicit\cpu\als.py:95: RuntimeWarning: OpenBLAS is configured to use 12 threads. It is highly recommended to disable its internal threadpool by setting the environment variable 'OPENBLAS_NUM_THREADS=1' or by calling 'threadpoolctl.threadpool_limits(1, "blas")'. Having OpenBLAS use a threadpool can lead to severe performance issues here.
  check_blas_config()


In [7]:
def get_recommendations(user_id, n):
    user_row = user2row[user_id]
    
    rec = model_als.recommend(user_row, user_item_matrix[user_row], n)[0]
    return np.array([col2item[col] for col in rec])

In [8]:
from evaluate import get_metrics

get_metrics(df_train, df_test, get_recommendations, k=10, alpha=0.2)

Evaluando usuarios: 100%|██████████| 15089/15089 [00:02<00:00, 5411.51it/s]

--- Métricas Globales de Evaluación ---
{
  "mean_recall": 0.030741613373723775,
  "mean_precision": 0.003332211376640862,
  "mean_ap (MAP)": 0.013230140030880522,
  "mean_ndcg": 0.017418443664565866,
  "mean_novelty": 8.482058093978155,
  "mean_diversity": 0.8362101471295906,
  "num_users_evaluated": 8913
}

--- Reporte de Fairness (Disparidad de Grupo) ---
{
  "delta_threshold": 0.2,
  "group_averages": {
    "Female": {
      "recall": 0.02252252252252252,
      "precision": 0.002293202293202293,
      "MAP": 0.010259772759772759,
      "nDCG": 0.013077422485910069,
      "novelty": 8.44868829551355,
      "diversity": 0.8361649326927105,
      "count": 2442
    },
    "Male": {
      "recall": 0.033843300880853036,
      "precision": 0.003724308453098439,
      "MAP": 0.014351085306115442,
      "nDCG": 0.0190566408084814,
      "novelty": 8.49465105454848,
      "diversity": 0.836227209972252,
      "count": 6471
    }
  },
  "disparity_reports": [
    {
      "pair": [
        "M

In [9]:
get_metrics(df_train, df_test, get_recommendations, k=10, alpha=0.2, use_age_group=True)

Evaluando usuarios: 100%|██████████| 15089/15089 [00:02<00:00, 6525.10it/s]

--- Métricas Globales de Evaluación ---
{
  "mean_recall": 0.031291789248462154,
  "mean_precision": 0.0033565124364803426,
  "mean_ap (MAP)": 0.013738023068726466,
  "mean_ndcg": 0.017925764530991085,
  "mean_novelty": 8.48303897854,
  "mean_diversity": 0.8364236573910506,
  "num_users_evaluated": 7478
}

--- Reporte de Fairness (Disparidad de Grupo) ---
{
  "delta_threshold": 0.2,
  "group_averages": {
    "25-34": {
      "recall": 0.032666163141993956,
      "precision": 0.0035120845921450153,
      "MAP": 0.014156386371265524,
      "nDCG": 0.018591743323805986,
      "novelty": 8.487263026509808,
      "diversity": 0.8366780890424522,
      "count": 5296
    },
    "35-44": {
      "recall": 0.026730637422892393,
      "precision": 0.002947224126113777,
      "MAP": 0.012897777342602565,
      "nDCG": 0.016178805170480447,
      "novelty": 8.459313680924774,
      "diversity": 0.8350823571836824,
      "count": 1459
    },
    "18-24": {
      "recall": 0.037444933920704845,
    